# About this Notebook
This notebook outlines the process of gathering data for a LLM application. This is important because it allows us to understand the ingredients of an LLM and thus decipher how and why it may be behaving in certain way. 

Pre training for properietary models is not transparent, so it is diffcult for us to know how and why models such as GPT-4, Claude, and Llama are behaving in certain ways. 

By gathering data from the internet, we can train our own models to understand how they are behaving. 


### Ingredients of an LLM 

1. Pretraining Data 
Garbage in, garbage out. In this notebook we will explore popular pretraining data sets and dig into preprocessing steps taken to ensur ehigh quality data is fed into our models. 

2. Vocabulary and Tokenizer
In order to builda model over a langauge, we have to first determine the vocabulary of the language we are modelling. We also need to break the stream of text into the corerect vocabulary units, referred to as tokenization. 

3. Learning Objective
What is the model being trained to do? Pretraining focuses on general skills such as syntax, semantics, reasoning etc such that it can handle any task we throw at it, even if it was not explicitly trained on it. Therefore training objectives should be sufficiently general to capture all these skills. 

4. Architecture
What is the internal architecture of the model? How does it process information? The arhcitecture refers to the components of the model, how they connnect and interact with each other and how they process input. Each architecture has its own inductive bias, a set of assumptions made about the data and tasks it will be used for. Biaising the model towards certain types of solutions can lead to better performance on certain tasks. 

5. Fine Tuning. 
The above described is the base model. We can then fine tune the model using supervised instruction fine tuning, RLHF, domain adaptive or task adaptive continured pretraining, so that the model is better attuned to specific domains and tasks.

# 
# ![Base Models and Derivatives](images/basemodels_and_derivatives.png)
 Base models are pretrained on large amounts of data and can be fine-tuned into derivative models for specific tasks and domains
# 


### Popular Pre Training Datasets
# ![Popular Pre Training Datasets](images/popular_pretraining_datasets.png)


### Exploring content of pre training datasets

In [2]:
from datasets import load_dataset

# Load the dataset
realnewslike = load_dataset("allenai/c4", "realnewslike", streaming=True, split="train")

# Iterate over the dataset
for i, example in enumerate(realnewslike):
    if "Iceland" in example["text"]:
        print(example)
    if i == 10000:  # Limit to 10,000 iterations for demonstration
        break

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'text': 'Kerth, 67, was selected from 164 applicants to be the organization’s new chief executive. He sits on the board of the Sacramento Municipal Utility District and is a scion of the Kerth family who founded the American Ice Co. and the Iceland skating rink in North Sacramento. He is responsible for the design of ice rinks in what’s now called Sleep Train Arena in Sacramento and an outdoor rink at Squaw Valley’s High Camp in 1990.\nVeteran nonprofit leader Pam Saltenberger, who served as Sacramento Habitat’s interim CEO for about six months, said she and other members of the executive search team felt that Kerth’s experience in the political arena would help the organization lead and navigate discussions about how governments and nonprofits can work together to ensure there’s adequate housing for low-income residents.\nThe loss of government funding is not the only challenge for the Sacramento Habitat, which has a budget of $5.9 million for the fiscal year ending in June. Nationwi

In [4]:
import ssl
import nltk

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jamespotter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
from collections import Counter
import re
from nltk.corpus import stopwords
import nltk


# Download stopwords if not already downloaded
try:
    stop_words = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

# Initialize counter
word_freq = Counter()

# Process first 10000 documents
for i, example in enumerate(realnewslike):
    # Convert to lowercase and split into words
    words = example["text"].lower().split()
    
    # Remove stopwords and count remaining words
    content_words = [word for word in words if word not in stop_words]
    word_freq.update(content_words)
    
    if i == 10000:  # Limit to 10,000 documents for demonstration
        break

# Print the 20 most common words
most_common_first_pass = []
print("20 most common words:")
for word, count in word_freq.most_common(20):
    print(f"{word}: {count}")
    most_common_first_pass.append(word)
    



20 most common words:
said: 10766
one: 9141
new: 9111
also: 8372
would: 7899
two: 5796
first: 5732
people: 5603
said.: 5512
like: 5465
last: 5250
could: 4663
time: 4502
—: 4475
get: 4277
many: 3951
even: 3881
make: 3776
–: 3696
may: 3689


In [6]:
# The most common stop words aren't representative of the topics in the data - so we will omit them and check for actual topics that occur. 
# Initialize counter for topic analysis
topic_freq = Counter()

# Process first 10000 documents, looking for meaningful topics
for i, example in enumerate(realnewslike):
    # Convert to lowercase and tokenize
    text = example["text"].lower()
    
    # Remove special characters and extra whitespace
    text = re.sub(r'[^\w\s]', ' ', text)
    text = ' '.join(text.split())
    
    # Split into words and remove stopwords
    words = text.split()
    content_words = [word for word in words 
                    if word not in stop_words 
                    and len(word) > 3 
                    and not word.isdigit() 
                    and word not in most_common_first_pass
                    and not any(c in word for c in ['—', '–'])] 
    
    # Count meaningful words that could represent topics
    topic_freq.update(content_words)
    
    if i == 10000:
        break

# Print the 20 most common meaningful topics
print("\n20 most common topics:")
for word, count in topic_freq.most_common(20):
    print(f"{word}: {count}")



20 most common topics:
year: 8716
years: 5273
state: 4518
well: 3862
three: 3841
back: 3661
world: 3616
company: 3581
work: 3467
city: 3459
much: 3245
million: 3212
government: 3175
high: 3164
take: 3131
president: 3128
home: 3125
made: 3093
school: 3058
since: 3007


In [7]:
# again, this is getting better but we can do better. We will only choose nouns.
# Initialize counter for topic analysis with only nouns
topic_freq = Counter()

# Process first 10000 documents, looking for meaningful topics
for i, example in enumerate(realnewslike):
    # Convert to lowercase and tokenize
    text = example["text"].lower()
    
    # Remove special characters and extra whitespace 
    text = re.sub(r'[^\w\s]', ' ', text)
    text = ' '.join(text.split())
    
    # Split into words and remove stopwords
    words = text.split()
    
    # Tag parts of speech
    pos_tags = nltk.pos_tag(words)
    
    # Only keep nouns (NN = noun singular, NNS = noun plural, NNP = proper noun singular, NNPS = proper noun plural)
    content_words = [word for word, pos in pos_tags 
                    if pos.startswith('NN')
                    and word not in stop_words
                    and len(word) > 3
                    and not word.isdigit()
                    and word not in most_common_first_pass
                    and not any(c in word for c in ['—', '–'])]
    
    # Count meaningful nouns that could represent topics
    topic_freq.update(content_words)
    
    if i == 10000:
        break

# Print the 20 most common meaningful noun topics
print("\n20 most common noun topics:")
for word, count in topic_freq.most_common(20):
    print(f"{word}: {count}")





20 most common noun topics:
year: 8716
years: 5273
state: 4518
world: 3616
company: 3581
city: 3459
government: 3175
president: 3128
school: 3058
home: 2975
team: 2923
game: 2895
percent: 2855
week: 2737
part: 2727
business: 2697
season: 2679
country: 2358
life: 2352
group: 2255


In [8]:
# Print the 20 least common meaningful noun topics
print("\n20 least common noun topics:")
for word, count in topic_freq.most_common()[:-21:-1]:
    print(f"{word}: {count}")


20 least common noun topics:
husak: 1
montgomerie: 1
banqueting: 1
mallaghan: 1
traum: 1
giesler: 1
wherley: 1
bock: 1
votech: 1
purvis: 1
lavash: 1
djan: 1
noshe: 1
logbooks: 1
partnerwere: 1
esquino: 1
learjet: 1
domene: 1
ávila: 1
yihai: 1


### Synthetic Data 
Cosmopedia is a synthetic dataset located in Huggingface.

In [9]:
from datasets import load_dataset
import random

# Load the Cosmopedia-100K dataset
dataset = load_dataset("HuggingFaceTB/cosmopedia-100k")

# Define some varied prompts to test
varied_prompts = [
    "Write a scientific article about black holes",
    "Compose a historical essay about ancient Rome",
    "Create a technical guide about machine learning",
    "Write a philosophical discussion about consciousness",
    "Explain quantum physics for beginners"
]

# Look at examples with different prompts
print("\nExamining Cosmopedia synthetic data samples with varied prompts:\n")

# Get random samples and pair with different prompts
samples = dataset['train'].select(random.sample(range(len(dataset['train'])), 5))

for i, (example, prompt) in enumerate(zip(samples, varied_prompts)):
    print(f"\nExample {i+1}:")
    print(f"Original Prompt: {example['prompt']}")
    print(f"Alternative Prompt: {prompt}")
    print(f"\nGenerated text: {example['text'][:500]}...")  # Show first 500 chars
    print("\n" + "="*80)

# Analyze dataset statistics
print("\nDataset Statistics:")
print(f"Total number of examples: {len(dataset['train'])}")
print(f"Average text length: {sum(len(ex['text']) for ex in dataset['train'].select(range(100)))/100:.0f} characters")

# Analyze prompt diversity
unique_prompts = set(example['prompt'] for example in dataset['train'].select(range(1000)))
print(f"Number of unique prompts in first 1000 examples: {len(unique_prompts)}")



Examining Cosmopedia synthetic data samples with varied prompts:


Example 1:
Original Prompt: Here is an extract from a webpage: "My vocabulary is not the most extensive, but I try to learn new words on a regular basis. And I succeed. I do not always succeed in retaining my new-found knowledge, though. I struggled to recall the meaning of “obeisance” (pronounced oh-bey-suh ns) a couple of days ago; my best ‘recollection’ of the meaning of the word was ‘deferential attitude.’ That’s relatively close, but not close enough. The word is a noun referring to a gesture or body movement expressing deep respect or deferential courtesy. I’m ruminating on how to remember what I learn.
Subscribe via Email
-
Recent Posts
Archives
Search on this Site
Categories
AR Writer Blogs/ Websites/Resources
- Writer Beware blog
- Considerable Opinions, Millie Gore Lancaster
- Wordcraft, Elizabeth Foster
- Southern Story Lady, Madelyn Young
- John Achor's site
- Nancy Smith Gibson's site
- It Just So Happens,

### Training Data Preprocessing
Once our data is collected, we need to filter and clean it by running it through a preproccessing pipeline. 

Data preprocessing is UNGLAMOUROUS and UNDERAPPRECIATED part of the LLM pipeline - yet is the most important. 

Spending more effort and resources on this step will yield the best results for our models.

As the data sets are very large, any preprocessing step should also be done very efficiently - ideally linear time. 

<image src="../images/data_collection_pre_processing_pipeline.png" width="800"></image>



## Each Step in detail

 ### 1. Data filtering and cleaning 
majority of extracted html text is jibberish. There is also a lot of pornographic content, toxic and hateful language on the web as well. 

Data from common crawl is made available in both html and wet formats. Wet often omits important information whilst html has a lot of data noise. 

We can use the JUSTTEXT library to reliably remove boiler plate text from html documents. 




In [10]:
# %pip install justext

import requests
import justext

response = requests.get("https://en.wikipedia.org/wiki/Toronto_Transit_Commission")
text = justext.justext(response.content, justext.get_stoplist("English"))
for content in text:
  if content.is_boilerplate:
    print(content.text)

Jump to content
Main menu
Main menu
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors learn more
Contributions
Talk
Contents
(Top)
1History
1.1Finances
2Operations
2.1Buses
2.2Subway
2.2.1Projects under construction
2.2.2Future plans
2.3Streetcars
3Services
3.1Fares
3.2Schedules and route information
3.3Connecting transit
3.3.1Connecting GTA transit agencies
3.3.2Connecting inter-city transit agencies
3.4Cellular and Wi-Fi connectivity
3.4.1Subway wireless services
3.5Accessibility
4Infrastructure
4.1Stations, stops and terminals
4.2Headquarters and facilities
4.3Commuter parking lots
5Safety
5.1Safety programs
5.1.1Crisis Link
5.1.2ThisIsWhere initiative and SafeTTC mobile app
5.2Transit Enforcement Unit
5.2.1Bylaws enforced
5.3Communications
5.3.1OneStop me

Justtext happens to be quite aggressive in removing content, but this is generally okay for cleaning pretrained datasets since there is an abundance of text available. 

Some alternative libraries include: 
1. dragnet

2. html2text 

3. inscriptis 

4. newspaper

5. trafilatura

Dividing the extraction pipeline across multiple libraries can reduce the risl of resulting dataset beign affected by any bias by one of these libraries.

LLMs can also be used for computationally-efficient text extraction from web pages - however is very expensive to do so. 

In [11]:
%pip install dragnet
%pip install html2text
%pip install inscriptis
%pip install newspaper
%pip install trafilatura

import dragnet
import html2text
import inscriptis
import newspaper
import trafilatura

response = requests.get("https://en.wikipedia.org/wiki/Bigfoot")
content = response.content



